In [15]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, PILToTensor
import time
import csv
torch.set_num_threads(1)


In [16]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(input)
print(target)
output = loss(input, target)
output.backward()

tensor([[ 1.0858, -1.3264,  0.1847,  1.3124, -1.4966],
        [ 2.2375,  1.1114,  0.1468,  1.0942,  3.2918],
        [-1.2351,  1.2349, -0.6531, -0.1235,  1.6095]], requires_grad=True)
tensor([4, 3, 2])
tensor([[-1.1176,  0.3197, -0.5723, -0.5822,  2.2362],
        [ 0.6394,  0.0132, -0.6417,  2.1846,  0.6606],
        [ 2.1773, -0.9261,  1.7127, -0.1169,  0.3892]], requires_grad=True)
tensor([[0.3596, 0.3287, 0.1758, 0.0905, 0.0454],
        [0.4407, 0.0721, 0.0616, 0.0760, 0.3496],
        [0.2337, 0.1936, 0.1586, 0.2746, 0.1395]])


In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [18]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    # transform=PILToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [19]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [21]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [22]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


In [23]:
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [24]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        # print("pred: ",pred.shape)
        # print("y:",y.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [25]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [26]:
epochs = 3
for datasize in [7500,15000,30000,60000]:
    model = NeuralNetwork().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    for epoch in range(epochs):
        subset_training_data = Subset(training_data, range(datasize))
        train_dataloader_subset = DataLoader(subset_training_data, batch_size=batch_size)
        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch+1} took {epoch_duration} seconds")
        with open("datasize_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, batch_size, datasize, accuracy, test_duration])

print("Done!")

Epoch 1
-------------------------------
loss: 2.314739  [   64/ 7500]
loss: 2.293372  [ 6464/ 7500]
Test Error: 
 Accuracy: 15.7%, Avg loss: 2.287916 

Epoch 1 took 0.8639187579974532 seconds
Epoch 2
-------------------------------
loss: 2.296772  [   64/ 7500]
loss: 2.277835  [ 6464/ 7500]
Test Error: 
 Accuracy: 30.0%, Avg loss: 2.269179 

Epoch 2 took 0.8607300380244851 seconds
Epoch 3
-------------------------------
loss: 2.279353  [   64/ 7500]
loss: 2.262231  [ 6464/ 7500]
Test Error: 
 Accuracy: 40.7%, Avg loss: 2.250628 

Epoch 3 took 0.8625829240772873 seconds
Epoch 1
-------------------------------
loss: 2.296216  [   64/15000]
loss: 2.290930  [ 6464/15000]
loss: 2.275679  [12864/15000]
Test Error: 
 Accuracy: 15.7%, Avg loss: 2.276729 

Epoch 1 took 1.730560079915449 seconds
Epoch 2
-------------------------------
loss: 2.271473  [   64/15000]
loss: 2.266398  [ 6464/15000]
loss: 2.245050  [12864/15000]
Test Error: 
 Accuracy: 29.1%, Avg loss: 2.247817 

Epoch 2 took 1.723268

In [27]:
epochs = 3
for bsize in [32,64,128,256]:
    model = NeuralNetwork().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    for epoch in range(epochs):
        train_dataloader_subset = DataLoader(training_data, batch_size=bsize)
        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch} took {epoch_duration} seconds")
        with open("minibatch_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, bsize, 60000, accuracy, test_duration])

print("Done!")

Epoch 1
-------------------------------
loss: 2.294715  [   32/60000]
loss: 2.287019  [ 3232/60000]
loss: 2.273346  [ 6432/60000]
loss: 2.254163  [ 9632/60000]
loss: 2.237249  [12832/60000]
loss: 2.227892  [16032/60000]
loss: 2.198553  [19232/60000]
loss: 2.169685  [22432/60000]
loss: 2.176881  [25632/60000]
loss: 2.116488  [28832/60000]
loss: 2.049807  [32032/60000]
loss: 2.026099  [35232/60000]
loss: 2.141241  [38432/60000]
loss: 2.037857  [41632/60000]
loss: 1.980370  [44832/60000]
loss: 1.976276  [48032/60000]
loss: 2.046535  [51232/60000]
loss: 1.921551  [54432/60000]
loss: 1.848428  [57632/60000]
Test Error: 
 Accuracy: 48.2%, Avg loss: 1.845236 

Epoch 0 took 7.686414301977493 seconds
Epoch 2
-------------------------------
loss: 1.841804  [   32/60000]
loss: 1.872982  [ 3232/60000]
loss: 1.789373  [ 6432/60000]
loss: 1.741879  [ 9632/60000]
loss: 1.680971  [12832/60000]
loss: 1.741491  [16032/60000]
loss: 1.555198  [19232/60000]
loss: 1.532688  [22432/60000]
loss: 1.540516  [25

In [28]:
epochs = 3
for bsize in [32,64,128,256]:
    model = NeuralNetwork().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    for epoch in range(epochs):
        train_dataloader_subset = DataLoader(training_data, batch_size=bsize)
        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch} took {epoch_duration} seconds")
        with open("minibatch_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, bsize, 60000, accuracy, test_duration])

print("Done!")

Epoch 1
-------------------------------
loss: 2.292461  [   32/60000]
loss: 2.299770  [ 3232/60000]
loss: 2.278650  [ 6432/60000]
loss: 2.267081  [ 9632/60000]
loss: 2.247157  [12832/60000]
loss: 2.237582  [16032/60000]
loss: 2.197761  [19232/60000]
loss: 2.174819  [22432/60000]
loss: 2.169621  [25632/60000]
loss: 2.139012  [28832/60000]
loss: 2.073025  [32032/60000]
loss: 2.052849  [35232/60000]
loss: 2.153680  [38432/60000]
loss: 2.039295  [41632/60000]
loss: 2.002890  [44832/60000]
loss: 2.009363  [48032/60000]
loss: 2.046635  [51232/60000]
loss: 1.953857  [54432/60000]
loss: 1.887060  [57632/60000]


In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>